
# CheckPoint 4 — COP30 (Belém) — Comparação (BF vs D&C vs DP)

Preencha a seção de **Composição do Grupo** abaixo e execute as células em ordem.



## Composição do Grupo
- Nome 1 — RA: _______
- Nome 2 — RA: _______
- Nome 3 — RA: _______
- Nome 4 — RA: _______


In [ ]:

import time, math
import matplotlib.pyplot as plt

from knapsack import gen_projects, knapsack_bruteforce, knapsack_meet_in_the_middle, knapsack_dp, QUALITY_REGISTRY as KNAPSACK_QREG
from maxsubarray import gen_days, maxsub_bruteforce, maxsub_divide_conquer, kadane, QUALITY_REGISTRY as MAXSUB_QREG

%matplotlib inline



## Desafio 1 — Mochila 0/1 (Seleção de Projetos)
Rodamos as três abordagens e validamos a correção em uma instância pequena; em seguida, benchmark no intervalo **(10..25)** como no enunciado.


In [ ]:

# Checagem de corretude rápida (instância pequena)
projects = [(3,25), (4,35), (7,70), (8,80)]
budget = 10
bf_b, _ = knapsack_bruteforce(projects, budget)
mi_b, _ = knapsack_meet_in_the_middle(projects, budget)
dp_b, _ = knapsack_dp(projects, budget)
assert bf_b == mi_b == dp_b, "Soluções diferentes no Desafio 1!"
print("OK — correção verificada (Desafio 1).")


In [ ]:

# Benchmark do Desafio 1 — n = 10..25 (aderente ao enunciado)
def benchmark_knapsack_assignment(ns=(10,12,14,16,18,20,22,24,25), repeats=1, budget=60):
    times_bf, times_mi, times_dp = [], [], []
    for n in ns:
        projects = gen_projects(n, seed=2025 + n)
        tb, tm, td = [], [], []

        # Warm-up fora da cronometragem
        try:
            if n <= 22:
                _ = knapsack_bruteforce(projects, budget)
        except ValueError:
            pass
        _ = knapsack_meet_in_the_middle(projects, budget)
        _ = knapsack_dp(projects, budget)

        for _ in range(repeats):
            # Brute Force (apenas até n<=22)
            if n <= 22:
                try:
                    t0 = time.perf_counter()
                    knapsack_bruteforce(projects, budget)
                    tb.append(time.perf_counter() - t0)
                except ValueError:
                    tb.append(float('nan'))
            else:
                tb.append(float('nan'))

            # MiTM
            t0 = time.perf_counter()
            knapsack_meet_in_the_middle(projects, budget)
            tm.append(time.perf_counter() - t0)

            # DP
            t0 = time.perf_counter()
            knapsack_dp(projects, budget)
            td.append(time.perf_counter() - t0)

        def mean_ms(vs):
            vals = [v for v in vs if not (isinstance(v,float) and math.isnan(v))]
            return (sum(vals)/len(vals)*1000) if vals else float('nan')

        times_bf.append(mean_ms(tb))
        times_mi.append(mean_ms(tm))
        times_dp.append(mean_ms(td))
    return ns, times_bf, times_mi, times_dp

ns, tbf, tmi, tdp = benchmark_knapsack_assignment()

plt.figure()
plt.plot(ns, tbf, marker="o", label="Brute Force (se aplicável)")
plt.plot(ns, tmi, marker="o", label="Meet-in-the-Middle")
plt.plot(ns, tdp, marker="o", label="DP bottom-up")
plt.xlabel("n (projetos)")
plt.ylabel("Tempo (ms)")
plt.title("Desafio 1 — Desempenho (n = 10..25)")
plt.legend()
plt.show()



## Desafio 2 — Máxima Soma Contígua (Janela de Qualidade do Ar)
Validamos a correção e rodamos o benchmark em **n = 200, 400, 800, 1200, 1600** como no enunciado.


In [ ]:

# Checagem de corretude (instância pequena)
arr = [3, -2, 5, -1, -4, 6, -3]
bf_v, _, _ = maxsub_bruteforce(arr)
dc_v, _, _ = maxsub_divide_conquer(arr)
kd_v, _, _ = kadane(arr)
assert bf_v == dc_v == kd_v, "Soluções diferentes no Desafio 2!"
print("OK — correção verificada (Desafio 2).")


In [ ]:

# Benchmark do Desafio 2 — ns exigidos
def benchmark_maxsub_assignment(ns=(200,400,800,1200,1600), repeats=1):
    t_bf, t_dc, t_kd = [], [], []
    for n in ns:
        arr = gen_days(n, seed=3000 + n)
        tb, td, tk = [], [], []

        # Warm-up
        try:
            if n <= 600:
                _ = maxsub_bruteforce(arr)
        except ValueError:
            pass
        _ = maxsub_divide_conquer(arr)
        _ = kadane(arr)

        for _ in range(repeats):
            # Brute Force até 600
            if n <= 600:
                try:
                    t0 = time.perf_counter()
                    maxsub_bruteforce(arr)
                    tb.append(time.perf_counter() - t0)
                except ValueError:
                    tb.append(float('nan'))
            else:
                tb.append(float('nan'))

            # Divide & Conquer
            t0 = time.perf_counter()
            maxsub_divide_conquer(arr)
            td.append(time.perf_counter() - t0)

            # Kadane
            t0 = time.perf_counter()
            kadane(arr)
            tk.append(time.perf_counter() - t0)

        def mean_ms(vs):
            vals = [v for v in vs if not (isinstance(v,float) and math.isnan(v))]
            return (sum(vals)/len(vals)*1000) if vals else float('nan')

        t_bf.append(mean_ms(tb))
        t_dc.append(mean_ms(td))
        t_kd.append(mean_ms(tk))
    return ns, t_bf, t_dc, t_kd

ns2, tbf2, tdc2, tkd2 = benchmark_maxsub_assignment()

plt.figure()
plt.plot(ns2, tbf2, marker="o", label="Brute Force (se aplicável)")
plt.plot(ns2, tdc2, marker="o", label="Divide & Conquer")
plt.plot(ns2, tkd2, marker="o", label="Kadane (DP)")
plt.xlabel("n (dias)")
plt.ylabel("Tempo (ms)")
plt.title("Desafio 2 — Desempenho (200..1600)")
plt.legend()
plt.show()



## Explicações e Conexões
- **Desafio 1 (Mochila):** Brute Force (baseline), Meet‑in‑the‑Middle (2^{n/2}), DP 1D por capacidade com reconstrução.  
- **Desafio 2 (Máx. Soma):** Brute Force (O(n²)), Divide & Conquer (O(n log n)), **Kadane (O(n))**.

## Conclusões
- DP domina quando aplicável (Mochila e Kadane), garantindo melhor escalabilidade.  
- Limites do brute force evitam travamentos; resultados validados mutuamente com `assert`.
